# 🤖 AtiendeSenas - Entrenamiento VideoMAE en Google Colab

**Proyecto**: Reconocimiento de Lengua de Señas con VideoMAE

**Autor**: Rafael Ovalle - Tesis UNAB

**Datasets disponibles**:
- WLASL100: 100 clases, ~1,000 videos
- WLASL300: 300 clases, ~2,790 videos

---

## 📋 Índice
1. Configuración inicial
2. Clonar repositorio
3. Instalar dependencias
4. Verificar datasets
5. Entrenamiento
6. Evaluación
7. Visualización de resultados

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1️⃣ Configuración Inicial

### Verificar GPU disponible

In [ ]:
import torch
import sys

print("Python version:", sys.version)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU Memory:", torch.cuda.get_device_properties(0).total_memory / 1024**3, "GB")
else:
    print("⚠️ WARNING: GPU no disponible. El entrenamiento será MUY lento.")
    print("Ve a Runtime > Change runtime type > GPU")

## 2️⃣ Clonar Repositorio

Clona tu repositorio de GitHub con el código y datasets.

In [ ]:
# Configurar Git (opcional, para commits)
!git config --global user.email "your.email@example.com"
!git config --global user.name "Your Name"

# Clonar el repositorio
!git clone https://github.com/Ov4llezz/AtiendeSenas-MVP.git

# Cambiar al directorio del proyecto
%cd AtiendeSenas-MVP

# Verificar que estamos en el directorio correcto
!pwd
!ls -la



%cd /content
!rm -rf AtiendeSenas-MVP

!git clone https://github.com/Ov4llezz/AtiendeSenas-MVP.git

%cd /content/AtiendeSenas-MVP
!ls

## 3️⃣ Instalar Dependencias

Instala todas las librerías necesarias.

In [ ]:
# Instalar dependencias desde requirements.txt si existe
!pip install -q transformers==4.36.0
!pip install -q torch torchvision torchaudio
!pip install -q opencv-python-headless
!pip install -q scikit-learn
!pip install -q matplotlib seaborn
!pip install -q tensorboard
!pip install -q tqdm

print("✅ Dependencias instaladas correctamente")

## 4️⃣ Verificar Datasets

Verifica que los datasets estén correctamente configurados.

In [ ]:
# Ejecutar script de verificación
!python scripts/verify_datasets.py

### Revisar estructura de datos manualmente

In [ ]:
import os

def count_videos(dataset_path):
    """Cuenta videos en cada split"""
    splits = ['train', 'val', 'test']
    print(f"\n📁 Dataset: {dataset_path}")
    print("=" * 50)

    for split in splits:
        split_path = os.path.join(dataset_path, 'dataset', split)
        if os.path.exists(split_path):
            videos = [f for f in os.listdir(split_path) if f.endswith('.mp4')]
            print(f"{split:5s}: {len(videos):4d} videos")
        else:
            print(f"{split:5s}: ❌ No existe")

# Verificar ambos datasets
count_videos('data/wlasl100')
count_videos('data/wlasl300')

## 5️⃣ Entrenamiento

### Configuración de Hiperparámetros

In [ ]:
# ========== CONFIGURACIÓN ==========
# Cambia estos valores según tus necesidades

DATASET = "wlasl100"  # "wlasl100" o "wlasl300"
BATCH_SIZE = 16       # Reduce si tienes problemas de memoria (ej: 8, 12)
MAX_EPOCHS = 30       # Número de epochs
LEARNING_RATE = 1e-4  # Learning rate
PATIENCE = 5          # Early stopping patience

print(f"""\n{'='*60}
CONFIGURACIÓN DE ENTRENAMIENTO
{'='*60}
Dataset:          {DATASET}
Batch size:       {BATCH_SIZE}
Max epochs:       {MAX_EPOCHS}
Learning rate:    {LEARNING_RATE:.1e}
Patience:         {PATIENCE}
{'='*60}
""")

### Entrenar Modelo

**Nota**: Este proceso puede tomar varias horas dependiendo del dataset y GPU.

In [ ]:
# Entrenar modelo
!python scripts/train.py \
    --dataset {DATASET} \
    --batch_size {BATCH_SIZE} \
    --max_epochs {MAX_EPOCHS} \
    --lr {LEARNING_RATE} \
    --patience {PATIENCE} \
    --class_weighted true \
    --early_stopping true

### Visualizar Training con TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs/

### 🚀 Entrenamiento Rápido (Para Testing)

Si solo quieres probar que todo funciona:

In [ ]:
# Entrenamiento de prueba rápido (solo 2 epochs)
!python scripts/train.py \
    --dataset wlasl100 \
    --batch_size 8 \
    --max_epochs 2 \
    --lr 1e-4

## 6️⃣ Evaluación

### Listar Modelos Entrenados

In [ ]:
# Ver todos los runs disponibles
!python scripts/test.py --list-runs

### Evaluar Modelo

Selecciona el Run ID del modelo que quieres evaluar (normalmente el más reciente es run-id 1)

In [ ]:
# Evaluar el modelo más reciente (run-id 1)
RUN_ID = 1

!python scripts/test.py --run-id {RUN_ID}

### Evaluar Checkpoint Específico

In [ ]:
# Buscar checkpoints disponibles
!ls -lh models/checkpoints/*/best_model.pt

In [ ]:
# Evaluar checkpoint específico
CHECKPOINT_PATH = "models/checkpoints/run_XXXXXX_XXXXXX/best_model.pt"  # Cambiar por tu path

!python scripts/test.py --checkpoint_path {CHECKPOINT_PATH}

## 7️⃣ Visualización de Resultados

### Ver Matriz de Confusión

In [ ]:
from IPython.display import Image, display
import glob

# Buscar la última matriz de confusión generada
confusion_matrices = sorted(glob.glob('evaluation_results/confusion_matrix_*.png'), reverse=True)

if confusion_matrices:
    print(f"📊 Mostrando: {confusion_matrices[0]}")
    display(Image(filename=confusion_matrices[0]))
else:
    print("❌ No se encontraron matrices de confusión. Ejecuta la evaluación primero.")

### Ver Métricas Detalladas

In [ ]:
import json
import glob

# Buscar el último reporte JSON
reports = sorted(glob.glob('evaluation_results/evaluation_report_*.json'), reverse=True)

if reports:
    with open(reports[0], 'r') as f:
        metrics = json.load(f)

    print("\n" + "="*60)
    print("📊 MÉTRICAS DE EVALUACIÓN")
    print("="*60)
    print(f"\nDataset: {metrics['dataset_info']['dataset']}")
    print(f"Muestras: {metrics['dataset_info']['test_samples']}")
    print(f"Clases: {metrics['dataset_info']['num_classes']}")

    print("\n--- Métricas Generales ---")
    print(f"Accuracy Total:     {metrics['overall_metrics']['total_accuracy']:.2f}%")
    print(f"Precision (Macro):  {metrics['overall_metrics']['precision_macro']:.2f}%")
    print(f"Recall (Macro):     {metrics['overall_metrics']['recall_macro']:.2f}%")
    print(f"F1-Score (Macro):   {metrics['overall_metrics']['f1_macro']:.2f}%")

    print("\n--- Top-K Accuracy ---")
    for k, v in metrics['top_k_accuracies'].items():
        print(f"{k}: {v:.2f}%")

    print("\n--- Top 10 Mejores Clases (por F1-Score) ---")
    per_class = metrics['per_class_metrics']
    sorted_classes = sorted(per_class.items(), key=lambda x: x[1]['f1_score'], reverse=True)[:10]
    for class_id, class_metrics in sorted_classes:
        print(f"Clase {class_id}: F1={class_metrics['f1_score']:.2f}%, "
              f"Precision={class_metrics['precision']:.2f}%, "
              f"Recall={class_metrics['recall']:.2f}%")

    print("\n--- Top 10 Peores Clases (por F1-Score) ---")
    sorted_classes = sorted(per_class.items(), key=lambda x: x[1]['f1_score'])[:10]
    for class_id, class_metrics in sorted_classes:
        print(f"Clase {class_id}: F1={class_metrics['f1_score']:.2f}%, "
              f"Precision={class_metrics['precision']:.2f}%, "
              f"Recall={class_metrics['recall']:.2f}%")

    print("="*60)
else:
    print("❌ No se encontraron reportes. Ejecuta la evaluación primero.")

### Graficar Curvas de Entrenamiento

In [ ]:
import matplotlib.pyplot as plt
from tensorboard.backend.event_processing import event_accumulator
import glob

def plot_training_curves(log_dir):
    """Grafica curvas de loss y accuracy desde TensorBoard logs"""
    ea = event_accumulator.EventAccumulator(log_dir)
    ea.Reload()

    # Obtener métricas disponibles
    tags = ea.Tags()['scalars']

    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Plot Loss
    if 'train_loss' in tags and 'val_loss' in tags:
        train_loss = [(s.step, s.value) for s in ea.Scalars('train_loss')]
        val_loss = [(s.step, s.value) for s in ea.Scalars('val_loss')]

        axes[0].plot([s[0] for s in train_loss], [s[1] for s in train_loss], label='Train Loss')
        axes[0].plot([s[0] for s in val_loss], [s[1] for s in val_loss], label='Val Loss')
        axes[0].set_xlabel('Epoch')
        axes[0].set_ylabel('Loss')
        axes[0].set_title('Training and Validation Loss')
        axes[0].legend()
        axes[0].grid(True)

    # Plot Accuracy
    if 'train_accuracy' in tags and 'val_accuracy' in tags:
        train_acc = [(s.step, s.value) for s in ea.Scalars('train_accuracy')]
        val_acc = [(s.step, s.value) for s in ea.Scalars('val_accuracy')]

        axes[1].plot([s[0] for s in train_acc], [s[1] for s in train_acc], label='Train Acc')
        axes[1].plot([s[0] for s in val_acc], [s[1] for s in val_acc], label='Val Acc')
        axes[1].set_xlabel('Epoch')
        axes[1].set_ylabel('Accuracy (%)')
        axes[1].set_title('Training and Validation Accuracy')
        axes[1].legend()
        axes[1].grid(True)

    plt.tight_layout()
    plt.show()

# Buscar el último run
runs = sorted(glob.glob('runs/run_*'), reverse=True)
if runs:
    print(f"📈 Graficando: {runs[0]}")
    try:
        plot_training_curves(runs[0])
    except Exception as e:
        print(f"⚠️ Error al graficar: {e}")
        print("Usa TensorBoard para ver las curvas de entrenamiento.")
else:
    print("❌ No se encontraron logs de entrenamiento.")

## 💾 Descargar Resultados

### Comprimir y Descargar Modelos

In [ ]:
# Comprimir modelos
!tar -czf models_checkpoints.tar.gz models/checkpoints/

# Comprimir resultados de evaluación
!tar -czf evaluation_results.tar.gz evaluation_results/

print("✅ Archivos comprimidos:")
!ls -lh *.tar.gz

# Para descargar, haz clic derecho en los archivos en el panel izquierdo

### Descargar Archivos Individuales

In [ ]:
from google.colab import files

# Descargar el mejor modelo
best_models = glob.glob('models/checkpoints/*/best_model.pt')
if best_models:
    files.download(best_models[0])

# Descargar matriz de confusión
confusion_matrices = glob.glob('evaluation_results/confusion_matrix_*.png')
if confusion_matrices:
    files.download(confusion_matrices[0])

# Descargar reporte de evaluación
reports = glob.glob('evaluation_results/evaluation_report_*.json')
if reports:
    files.download(reports[0])

## 🔧 Utilidades

### Monitorear Uso de GPU

In [ ]:
!nvidia-smi

### Limpiar Memoria GPU

In [ ]:
import gc
import torch

# Limpiar cache de GPU
gc.collect()
torch.cuda.empty_cache()

print("✅ Memoria GPU liberada")

### Montar Google Drive (Para guardar resultados)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Copiar checkpoints a Drive
!cp -r models/checkpoints /content/drive/MyDrive/AtiendeSenas_Checkpoints/
!cp -r evaluation_results /content/drive/MyDrive/AtiendeSenas_Results/

print("✅ Resultados copiados a Google Drive")

## ⚠️ Solución de Problemas

### Error: Out of Memory
- Reduce `BATCH_SIZE` (ej: de 16 a 8 o 12)
- Limpia memoria GPU ejecutando la celda de "Limpiar Memoria GPU"
- Reinicia el runtime: Runtime > Restart runtime

### Error: No GPU available
- Ve a Runtime > Change runtime type
- Selecciona GPU como Hardware accelerator
- Guarda y reconecta

### Sesión de Colab se desconecta
- Colab tiene límite de tiempo (12h para usuarios gratuitos)
- Guarda checkpoints frecuentemente
- Usa Google Drive para respaldos

### Videos no se cargan
- Verifica que los videos estén en el repositorio
- Ejecuta `verify_datasets.py` para diagnóstico
- Revisa los paths en `WLASLDataset.py`

## 📝 Notas Importantes

1. **Tiempo de Entrenamiento**:
   - WLASL100: ~2-4 horas (30 epochs)
   - WLASL300: ~6-12 horas (30 epochs)
   - Depende de la GPU disponible

2. **Checkpoints**:
   - Se guardan automáticamente cada 5 epochs
   - El mejor modelo se guarda en `best_model.pt`
   - Puedes resumir entrenamiento con `--resume`

3. **Early Stopping**:
   - Se detiene si no mejora en 5 epochs (configurable)
   - Basado en validation loss

4. **Recursos de Colab**:
   - GPU gratuita: T4 (16GB)
   - Límite de tiempo: 12 horas continuas
   - Guarda resultados frecuentemente

---

**¡Buena suerte con el entrenamiento! 🚀**